# Imports

In [168]:
import json

import folium
import numpy as np
import pandas as pd
from matplotlib import cm, colors

from data.referential.communes import Communes, CommunesGeojsonDictKey
from data.referential.departements import Departements, DepartementsGeojsonDictKey
from data.referential.regions import Regions, RegionsGeojsonDictKey

from data.clean.loyers import load_loyers_par_ville_df
from data.clean.prix_communes import load_prix_par_commune


# Configuration


In [169]:
regions_geojson_path = r"data/geo/regions.json"
map_provider = "Cartodb Positron"
arrondissements_paris_code_insee = [str(75100 + i) for i in range(1, 21, 1)]
code_insee_paris = "75056"

# Data loading

## Loading

In [170]:
with open(regions_geojson_path, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# valeurs_foncieres = load_valeurs_foncieres_df()
prix_par_commune = load_prix_par_commune()
loyers = load_loyers_par_ville_df()
insee_codes = load_insee_codes_communes_df()

## Cleaning

In [171]:
ile_de_france_geojson = {
    "type": "FeatureCollection",
    "features": [
        feature for feature in geojson_data['features'] if feature['properties']['nom'] == "Île-de-France"
    ]
}

insee_codes = insee_codes[insee_codes["Région"] == "ILE-DE-FRANCE"]

prix_par_commune = prix_par_commune[prix_par_commune["Annee"] == 2023]

# Data preparation

## Colour scale

In [172]:
prix_m2_moyen_min = prix_par_commune["Prixm2Moyen"].min()
prix_m2_moyen_max = prix_par_commune["Prixm2Moyen"].max()

log_transformed_values = np.log1p(prix_par_commune["Prixm2Moyen"])
log_min = log_transformed_values.min()
log_max = log_transformed_values.max()

print(f"Prix moyen m2 min: {prix_m2_moyen_min}")
print(f"Prix moyen m2 max: {prix_m2_moyen_max}")

norm = colors.Normalize(vmin=log_min, vmax=log_max)
color_map = cm.ScalarMappable(norm=norm, cmap="coolwarm")


Prix moyen m2 min: 400.0
Prix moyen m2 max: 14000.0


## Data per city

In [178]:
cities_statistics = {}

for index, city in insee_codes.copy().iterrows():
    prix_m2_moyen = np.nan
    if city["Code INSEE"] in pd.unique(prix_par_commune["Code INSEE"]):
        prix_m2_moyen = int(prix_par_commune[prix_par_commune["Code INSEE"] == city["Code INSEE"]]["Prixm2Moyen"].iloc[0])
    elif city["Code INSEE"] in arrondissements_paris_code_insee and code_insee_paris in pd.unique(prix_par_commune["Code INSEE"]):
        prix_m2_moyen = int(prix_par_commune[prix_par_commune["Code INSEE"] == code_insee_paris]["Prixm2Moyen"].iloc[0])
    city_data = {
        "population": int(city["Population"] * 1000),
        "prix_m2_moyen": prix_m2_moyen
    }
    cities_statistics[city["Code INSEE"]] = city_data

# Analyse

## Map background

In [189]:
ile_de_france_center = [48.85, 2.35]

# Create a folium map
map_ile_de_france = folium.Map(location=ile_de_france_center, zoom_start=9, tiles=map_provider)

# Add the Île-de-France GeoJSON layer
folium.GeoJson(
    ile_de_france_geojson,
    name="Île-de-France",
    style_function=lambda x: {
        'fillColor': 'transparent',  # No fill color
        'fillOpacity': 0,  # Make the fill transparent
        'color': 'red',  # Set the border color to red
        'weight': 2  # Set the thickness of the border
    }
).add_to(map_ile_de_france)

# Add layer control (optional)
folium.LayerControl().add_to(map_ile_de_france)

## Add cities

In [190]:
for index, city in insee_codes.copy().iterrows():
    price = "N/A"
    population = "N/A"
    if city["Code INSEE"] in cities_statistics.keys():
        value = cities_statistics[city["Code INSEE"]]["prix_m2_moyen"]
        price = f"{value}€"
        population = cities_statistics[city["Code INSEE"]]["population"]
        color = colors.rgb2hex(color_map.to_rgba(np.log1p(value)))
    else:
        color = "black"

    description = f"<b>{city['Commune']}</b><br>Prix moyen: {price}<br>Population: {population}"
    popup = folium.Popup(description, max_width=500)

    folium.Polygon(
        locations=city["geo_shape"]["coordinates"][0],
        color='blue',
        weight=1,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=popup
    ).add_to(map_ile_de_france)

## Display map

In [191]:
map_ile_de_france

In [192]:
map_ile_de_france.save("map_ile_de_france.html")